In [1]:
# Imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc

import os
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests

In [2]:
# Configure Chrome Webdriver

chrome_install = ChromeDriverManager().install()

In [3]:
# Initialize Chrome WebDriver
options = webdriver.ChromeOptions()
#options.add_argument('--headless')

options.add_argument(
    "--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 17_0 like Mac OS X) "
    "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1"
)

browser = webdriver.Chrome(options=options)

In [ ]:
# Setup search parameters
city = "Redondo Beach"
city_link = "Redondo Beach"
product = "Iphone"
min_price = 0
max_price = 5000
days_listed = 10

In [5]:
# Setup base URL
url = f'https://www.facebook.com/marketplace/{city}/search?query={product}&minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_listed}&exact=false'

# Visit website
browser.get(url)

# Close login pop-up
try:
    close_button = browser.find_element(By.XPATH, "//div[@aria-label='Close']")
    close_button.click()
    print("Login prompt closed successfully...")
except:
    print("Login prompt not closed correctly...")
    pass

Login prompt not closed correctly...


In [6]:
# Scroll down to load all results
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    
    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    
    print("Scrolled..")

print("Finished scrolling, all results loaded...")


Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Finished scrolling, all results loaded...


In [7]:
# Retrieve the HTML
html = browser.page_source

soup = BeautifulSoup(html, 'html.parser')

browser.close()

In [8]:
# Find link elements
links = soup.find_all('a')

# Filter on product keyword only
product_links = [
    link for link in links
    if product.lower() in link.text.lower()
    #if city_link.lower() in link.text.lower()
]

product_data = []

for product_link in product_links:
    url = product_link.get('href')
    if not url:
        continue
    text = '\n'.join(product_link.stripped_strings)
    product_data.append({'url': url, 'text': text})

print(len(product_links), "links matched")
print(len(product_data), "items collected")

product_data

614 links matched
614 items collected


[{'url': '/marketplace/item/1171625514926433/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$350\nIphone 12 pro max\nSt-Sylvère, QC'},
 {'url': '/marketplace/item/1764948837546297/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$800\niPhone 15 pro max\nLongueuil, QC'},
 {'url': '/marketplace/item/1413696196992922/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$850\niPhone 15 Pro Max\nSherbrooke, QC'},
 {'url': '/marketplace/item/823574900653610/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$700\niPhone 14 Pro Max 128GB\nBrossard, QC'},
 {'url': '/marketplace/item/1364465141731627/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$700\niPhone 15 pro Max\nMontréal, QC'},
 {'url': '/marketplace/item/1499241924675631/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$350\niPhon

In [9]:
# Create an empty list to store product data
extracted_data = []

for item in product_data:
    lines = item['text'].split('\n')

    # Regular expression to find numeric values
    numeric_pattern = re.compile('\d[\d,.]*')
    
    
    # Extracting prices
    # Iterate through lines to find the first line with numbers
    for line in lines:
        match = numeric_pattern.search(line)
        if match:    
            # Extract the first numeric value found
            price_str = match.group()
            # Convert price to float (handle commas)
            price = float(price_str.replace(',',''))
            break
            
    if price:
        print(f"Price extracted: {price}")
    else:
        print("price not found")

    # Extract title
    title = lines[-2]

    # Extract location
    location = lines[-1]

    # Add extracted data to a list of dictionaries
    extracted_data.append({
        'title': title,
        'price': price,
        'location': location,
        'url': re.sub(r'\?.*', '', item['url'])
    })

Price extracted: 350.0
Price extracted: 800.0
Price extracted: 850.0
Price extracted: 700.0
Price extracted: 700.0
Price extracted: 350.0
Price extracted: 725.0
Price extracted: 900.0
Price extracted: 250.0
Price extracted: 280.0
Price extracted: 250.0
Price extracted: 250.0
Price extracted: 895.0
Price extracted: 215.0
Price extracted: 700.0
Price extracted: 1500.0
Price extracted: 150.0
Price extracted: 200.0
Price extracted: 200.0
Price extracted: 260.0
Price extracted: 110.0
Price extracted: 850.0
Price extracted: 900.0
Price extracted: 400.0
Price extracted: 750.0
Price extracted: 200.0
Price extracted: 300.0
Price extracted: 400.0
Price extracted: 200.0
Price extracted: 420.0
Price extracted: 180.0
Price extracted: 60.0
Price extracted: 200.0
Price extracted: 395.0
Price extracted: 220.0
Price extracted: 700.0
Price extracted: 700.0
Price extracted: 160.0
Price extracted: 450.0
Price extracted: 480.0
Price extracted: 750.0
Price extracted: 675.0
Price extracted: 215.0
Price extra

<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
/var/folders/xn/w_xnr1cn1jq5m_6rgnybnsc40000gn/T/ipykernel_47688/1258939502.py:8: SyntaxWarning: invalid escape sequence '\d'
  numeric_pattern = re.compile('\d[\d,.]*')


In [10]:
extracted_data

[{'title': 'Iphone 12 pro max',
  'price': 350.0,
  'location': 'St-Sylvère, QC',
  'url': '/marketplace/item/1171625514926433/'},
 {'title': 'iPhone 15 pro max',
  'price': 800.0,
  'location': 'Longueuil, QC',
  'url': '/marketplace/item/1764948837546297/'},
 {'title': 'iPhone 15 Pro Max',
  'price': 850.0,
  'location': 'Sherbrooke, QC',
  'url': '/marketplace/item/1413696196992922/'},
 {'title': 'iPhone 14 Pro Max 128GB',
  'price': 700.0,
  'location': 'Brossard, QC',
  'url': '/marketplace/item/823574900653610/'},
 {'title': 'iPhone 15 pro Max',
  'price': 700.0,
  'location': 'Montréal, QC',
  'url': '/marketplace/item/1364465141731627/'},
 {'title': 'iPhone 13',
  'price': 350.0,
  'location': 'Trois-Rivières, QC',
  'url': '/marketplace/item/1499241924675631/'},
 {'title': 'iPhone 14 Pro Max',
  'price': 725.0,
  'location': 'Brossard, QC',
  'url': '/marketplace/item/1535626471016988/'},
 {'title': 'iPhone',
  'price': 900.0,
  'location': 'Cornwall, ON',
  'url': '/marketpla